In [74]:
# IMPORTING LIBRARIES
import random
import numpy as np

# CONFIG SETUP
suits = ['C','D','H','S']
suits = ["♠","♣","♥","♦" ]
ranks  = {  "J":3,
            "9":2,
            "A":1,"10":1,
            "K":0,"Q":0
}
delim = "|"
num_players= 4
starting_player = 1

In [75]:
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.power = ranks[rank]

    def show(self):
        return self.rank + delim + self.suit


    

In [76]:
class Player:
    def __init__(self, play_n, init_hand):
        self.player_num = play_n
        self.hand = sorted(init_hand, key = lambda card: -1*card.power)

        self.team = play_n%2

    def pcard_sort(self, pcards):
        return sorted(pcards, key = lambda x: -1*x['card'].power)

    def strategy_1(self, round_cards, trump = ""):
        if len(round_cards)==0:
            ### OPENING HAND
            sorted_hand = sorted(self.hand, key = lambda card: -1*card.power)

            if sorted_hand[0].rank == 'J':
                # If player has a J then play it
                play = sorted_hand[0]
            else:
                # Else play the lowest hand
                play = sorted_hand[-1]
            

        else:
            ### ONGOING ROUND
            reqd_suit = round_cards[0]['card'].suit
            sort_cards =sorted(round_cards, key = lambda play: -1*play['card'].power)


            top_card = sort_cards[0]
           
            vettus = self.pcard_sort([play for play in round_cards if play['card'].suit==trump])
            if len(vettus)>0: team_vettu = vettus[0]['player']%2==self.team
            else: team_vettu = False
            winning_team = (top_card['player']%2==0) or (team_vettu)
             

            # and (top_card[-1].power >= 2)
            # print(f"Team {self.team} winning: {winning_team}")

            if winning_team==self.team: func = max
            else: func = min

            # print(sort_cards)
            valid_cards = [card for card in self.hand if card.suit==reqd_suit]

            if len(valid_cards)!=0:
                # You HAVE a valid Suit Card
                if winning_team:
                    play = valid_cards[0]
                else:
                    play = valid_cards[-1]
                    
            else:
                # You DO NOT HAVE a valid Suit Card
                j = [card for card in self.hand if card.power==3]
                nonj = [card for card in self.hand if card.power<3]
                if winning_team:
                    if len(nonj)>0:
                        play = nonj[0]
                    else:
                        play = j[0]


                else:
                    if len(nonj)==0:
                        
                        play = j[0]
                    else:
                        play = nonj[-1]
                    
        self.hand.remove(play)
        return play
    

In [77]:
class Game:
    
    def __init__(self, num_players, show = False):
        ### STARTING THE GAME BY DIRTRIBUTING CARDS

        # Getting all cards set up
        cards = [[Card(suit, rank) for suit in suits] for rank in ranks.keys()]
        # Converting List of Lists into a single List
        cards = [i for j in cards for i in j]

        if num_players >= 5: cards = cards*2

        
        # Shuffling the Cards
        random.shuffle(cards)
        
        # Splitting it to num_players in a dict
        self.players = {k+1: Player(k+1, list(hand)) for k, hand in enumerate(np.array_split(cards,num_players))}

        if show: self.show_hands()

        self.rounds = {}


    def set_trump(self, trump):
        self.trump = trump
    
    def show_hands(self):
        print("Current Hand of each Player")
        for player_num, player in self.players.items():
            print(str(player_num) + ") " + ", ".join([card.show() for card in player.hand]))
            

    def play_round(self, start_player=1):
        round_no = len(self.rounds.keys()) + 1
        print(f"\nRound {round_no}")

        round_stats = {}
        
        all_players = list(range(1,num_players+1))
        player_index = all_players.index(start_player)
        play_order = all_players[player_index:] + all_players[:player_index]

        self.round_cards = []
        for p,play in enumerate(play_order):
            play_card = self.players[play].strategy_1(self.round_cards, self.trump)
            print(f"{p}) Player {play}: {play_card.show()}")

            
            self.round_cards.append({'player':play, 'card':play_card})

        self.rounds[round_no] = self.round_cards

    def next_round_player(self):
        latest_round = self.rounds[max(self.rounds.keys())]
        start_card = latest_round[0]['card']
        start_suit = start_card.suit
        start_rank = start_card.rank

        trumps = [play for play in latest_round if play['card'].suit==self.trump]
        vettu = len(trumps)>0
        if vettu:
            return sorted(trumps, key = lambda play: -1*play['card'].power)[0]['player']
        
        only_suit = sorted([play for play in latest_round if play['card'].suit==start_suit],
                           key = lambda play: -1*play['card'].power)
        return only_suit[0]['player'] 

            

In [82]:
game = Game(num_players, show = True)
game.set_trump(suits[1])
print(f"TRUMP = {game.trump}")

next_round_start = 1
for k in range(1,int(28/num_players)):
    game.play_round(next_round_start)
    # game.show_hands()
    next_round_start = game.next_round_player()


Current Hand of each Player
1) 9|♦, 10|♠, A|♠, 10|♦, A|♦, K|♣
2) J|♠, J|♣, 10|♣, A|♣, Q|♦, K|♠
3) J|♦, 9|♣, 10|♥, Q|♥, Q|♣, K|♦
4) J|♥, 9|♠, 9|♥, A|♥, K|♥, Q|♠
TRUMP = ♣

Round 1
0) Player 1: K|♣
1) Player 2: A|♣
2) Player 3: 9|♣
3) Player 4: Q|♠

Round 2
0) Player 3: J|♦
1) Player 4: K|♥
2) Player 1: A|♦
3) Player 2: Q|♦

Round 3
0) Player 3: K|♦
1) Player 4: A|♥
2) Player 1: 9|♦
3) Player 2: K|♠

Round 4
0) Player 1: 10|♦
1) Player 2: 10|♣
2) Player 3: Q|♣
3) Player 4: 9|♠

Round 5
0) Player 2: J|♠
1) Player 3: 10|♥
2) Player 4: 9|♥
3) Player 1: 10|♠

Round 6
0) Player 2: J|♣
1) Player 3: Q|♥
2) Player 4: J|♥
3) Player 1: A|♠
